In [1]:
import requests
import lxml
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
# import random
import re
import datetime

today=str(datetime.date.today())
print(today)

2023-01-11


In [2]:
url = "https://www.imdb.com/showtimes/"+today+"?ref_=sh_dt"
print(url)

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}
f = requests.get(url, headers = headers)

https://www.imdb.com/showtimes/2023-01-11?ref_=sh_dt


In [3]:
movies_lst = []
contr_lst = []
drt_lst = []
ur_lst = []
u_lst = []
ms_lst = []
url_lst = []
sl_lst = []
rv_lst = []
gr_lst = []

soup = BeautifulSoup(f.content, 'lxml')
each_div = soup.find_all(name='div',attrs={'class':'info'})

#get movie names, add them to 
for div in each_div:
    name = div.find(name='a', attrs={'title':re.compile(r"Showtimes for(\s\w+)?")})
    if name is not None:
        ttle = name.get_text()
    movies_lst.append(ttle)
    
    
    contrat = div.find('img')
    try:
        ctrt = contrat.get('title')
        contr_lst.append(ctrt)
    except:
        contr_lst.append('none')
    
    
    dur = div.find('time')
    try:
        if dur is not None:
            dtime = dur.get_text().replace(' min','')
        drt_lst.append(dtime)
    except:
        drt_lst.append('none')
        
    
    urate_f = div.find(name='strong', attrs={'itemprop':'ratingValue'})
    try:
        if urate_f is not None:
            urate = urate_f.get_text()
        ur_lst.append(urate)
    except:
        ur_lst.append('none')
    
    
    ucount = div.find(name='span', attrs={'itemprop':'ratingCount'})
    try:
        if ucount is not None:
            uc = ucount.get_text()
        u_lst.append(uc)
    except:
        u_lst.append('none')
        
    
    ms = div.find(name='span', attrs={'class':re.compile(r"metascore(\s\w+)?")})
    try:
        if ms is not None:
            mscore = ms.get_text().strip()
        ms_lst.append(mscore)
    except:
        ms_lst.append('none')   
    
    
    drl = div.find(name='a', attrs={'title':re.compile(r"Showtimes for(\s\w+)?")})
    if drl is not None:
        dl = drl.get('href').replace('/showtimes/','/').replace(today,'')
#         print(dl)
        strinfo = re.compile(r'\?ref_=\w*')
        durl = strinfo.sub('', dl).strip()
#         print(durl)
    url_lst.append('https://www.imdb.com'+durl)
    


#     try:
#         if drl is not None:
#             durl = ms.get('href')
#         url_lst.append(durl)
#     except:
#         url_lst.append('none')

#     time.sleep(random.random()*3)



In [4]:
d = {'title':movies_lst, 'content_rating':contr_lst, 'duration_min':drt_lst, 'users_rating':ur_lst, 'users_count':u_lst, 'metascore':ms_lst,'url':url_lst}
fi = pd.DataFrame(data = d)
fin = fi.drop_duplicates(['title'])

In [5]:
#Test: any repetitive items in the list?
fin.duplicated()
# set_lst = set(movies_lst)
# if len(set_lst) == len(movies_lst):
#     print('ok')
print(fin)

                                                 title content_rating  \
0                             A Man Called Otto (2022)          PG-13   
1                      Avatar: The Way of Water (2022)          PG-13   
2                                       Babylon (2022)              R   
3                Black Panther: Wakanda Forever (2022)          PG-13   
4                                       Corsage (2022)           none   
5                                         M3GAN (2022)          PG-13   
6                                         Plane (2023)              R   
7                  Puss in Boots: The Last Wish (2022)             PG   
8                                 Shin Ultraman (2022)           none   
9                                 The Fabelmans (2022)          PG-13   
10                                     The Menu (2022)              R   
11                                    The Whale (2022)              R   
12                                Violent Night (20

In [6]:
for ul in fin['url']:
    f1 = requests.get(ul, headers=headers)
    soup1 = BeautifulSoup(f1.content, 'lxml')
    
    try:
        sl = soup1.find(name='meta', attrs={'name':'description'})['content']
        sl_lst.append(sl)
    except:
        sl_lst.append('~~') #no description


fin.insert(6,'description', sl_lst)

print(fin)

                                                 title content_rating  \
0                             A Man Called Otto (2022)          PG-13   
1                      Avatar: The Way of Water (2022)          PG-13   
2                                       Babylon (2022)              R   
3                Black Panther: Wakanda Forever (2022)          PG-13   
4                                       Corsage (2022)           none   
5                                         M3GAN (2022)          PG-13   
6                                         Plane (2023)              R   
7                  Puss in Boots: The Last Wish (2022)             PG   
8                                 Shin Ultraman (2022)           none   
9                                 The Fabelmans (2022)          PG-13   
10                                     The Menu (2022)              R   
11                                    The Whale (2022)              R   
12                                Violent Night (20

In [7]:
for ul in fin['url']:
    f1 = requests.get(ul, headers=headers)
    soup1 = BeautifulSoup(f1.content, 'lxml')
    all_rev = soup1.find("script", {"type": "application/ld+json"})
    if all_rev is not None:
        revinfo = all_rev.contents[0]
        revjson = json.loads(revinfo)  # convert to json
        try:
            rev= revjson['review']
            rb = rev['reviewBody'].replace('\n\n',' ').replace('&apos;',"'").replace('&quot;','"')
            rn = rev['name']
            rv = rn +' - '+ rb
            rv_lst.append(rv)
        except:
            rv_lst.append('~')
        
        try:
            gr= revjson['genre']
            gr_lst.append(gr)
        except:
            gr_lst.append('~')
            
    else:
        rv_lst.append('~~')
        gr_lst.append('~~')
        
#             rb = rev['reviewBody'].replace('\n\n',' ').replace('&apos;',"'").replace('&quot;','"')
#             rn = rev['name']
#             rv = rn +' - '+ rb
#             rv_lst.append(rv)
#         except:
#             rv_lst.append('~~') #represents not yet reviewed

# print(rv_lst)

In [8]:
# for ul in fin['url']:
#     f1 = requests.get(ul, headers=headers)
#     soup1 = BeautifulSoup(f1.content, 'lxml')
#     all_rev = soup1.find("script", {"type": "application/ld+json"})
#     if all_rev is not None:
#         revinfo = all_rev.contents[0]
#         revjson = json.loads(revinfo)  # convert to json
#         try:
#             gr= revjson['genre']
#             print(str(gr))
# #             rb = rev['reviewBody'].replace('\n\n',' ').replace('&apos;',"'").replace('&quot;','"')
# #             rn = rev['name']
# #             rv = rn +' - '+ rb
# #             rv_lst.append(rv)
#         except:
#             print('~')
#     else:
#         print('~~')

In [9]:
fin.insert(7,'featured_review', rv_lst)
fin.insert(8,'genre', gr_lst)

In [10]:
print(fin)

                                                 title content_rating  \
0                             A Man Called Otto (2022)          PG-13   
1                      Avatar: The Way of Water (2022)          PG-13   
2                                       Babylon (2022)              R   
3                Black Panther: Wakanda Forever (2022)          PG-13   
4                                       Corsage (2022)           none   
5                                         M3GAN (2022)          PG-13   
6                                         Plane (2023)              R   
7                  Puss in Boots: The Last Wish (2022)             PG   
8                                 Shin Ultraman (2022)           none   
9                                 The Fabelmans (2022)          PG-13   
10                                     The Menu (2022)              R   
11                                    The Whale (2022)              R   
12                                Violent Night (20

In [11]:
fin.to_csv('final.csv')

In [13]:
fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 232
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            32 non-null     object
 1   content_rating   32 non-null     object
 2   duration_min     32 non-null     object
 3   users_rating     32 non-null     object
 4   users_count      32 non-null     object
 5   metascore        32 non-null     object
 6   description      32 non-null     object
 7   featured_review  32 non-null     object
 8   genre            32 non-null     object
 9   url              32 non-null     object
dtypes: object(10)
memory usage: 2.8+ KB
